In [2]:
import numpy as np 
import pandas as pd
import requests
from groq import Groq
from datetime import datetime
import calendar
import matplotlib.pyplot as plt
import math
from openai import OpenAI
import time

In [6]:
contracts = pd.read_csv("/home/xikron/Projects/misc/data/blacktesting/c16.csv")
contracts['Start Date']= pd.to_datetime(contracts['Start Date'])
contracts=contracts.drop_duplicates(subset='Award ID').sample(100, replace=False).sort_values(by='Start Date')
contracts.head()

,internal_id,Award ID,Recipient Name,Award Amount,Total Outlays,Description,Contract Award Type,def_codes,COVID-19 Obligations,COVID-19 Outlays,...,Infrastructure Outlays,Awarding Agency,Awarding Sub Agency,Start Date,End Date,recipient_id,prime_award_recipient_id,awarding_agency_id,agency_slug,generated_internal_id
287,37425564,W52P1J16C0014,LMT,2.705379e+08,NaN,IGF::OT::IGF,DEFINITIVE CONTRACT,NaN,NaN,NaN,...,NaN,Department of Defense,Department of the Army,2016-01-01,2022-02-28,3306c327-c59a-1581-3de2-01f8871c6ab0-C,NaN,1173.0,department-of-defense,CONT_AWD_W52P1J16C0014_9700_-NONE-_-NONE-
199,37216264,VA77016E0701,MCK,2.136064e+08,NaN,EXPRESS REPORT: PHARMACY PRIME VENDOR CMOP FY1...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Veterans Affairs,Department of Veterans Affairs,2016-01-01,2016-01-31,b54de748-ee64-a1e6-546e-e7007b054797-C,NaN,561.0,department-of-veterans-affairs,CONT_AWD_VA77016E0701_3600_VA797P12D0001_3600
444,25893090,N0003016C0045,LMT,9.808436e+07,NaN,IGF::OT::IGF UNDEFINITIZED CONTRACT ACTION FOR...,DEFINITIVE CONTRACT,['Q'],NaN,NaN,...,NaN,Department of Defense,Defense Contract Management Agency,2016-01-01,2021-02-24,0701db1f-8045-4e60-83f7-5fd066cdf38d-C,NaN,1173.0,department-of-defense,CONT_AWD_N0003016C0045_9700_-NONE-_-NONE-
129,3655883,0393,BA,4.383449e+07,NaN,CONTRACTOR LOGISTICS SUPPORT FOR THE C-32 AND ...,DELIVERY ORDER,NaN,NaN,NaN,...,NaN,Department of Defense,Defense Contract Management Agency,2016-01-01,2016-06-30,22d282c6-4508-7d30-97f1-7401ea73b12a-C,NaN,1173.0,department-of-defense,CONT_AWD_0393_9700_F3365701D0013_9700
440,15066767,FA865016C7603,BAESY,3.286608e+07,1426187.23,IGF::OT::IGF INSIGHT TRANSITION,DEFINITIVE CONTRACT,"['N', 'Q']",-454516.15,654516.15,...,NaN,Department of Defense,Department of the Air Force,2016-01-25,2024-01-30,3aadb63a-54fd-526c-ec7e-f2d1e63d8112-C,NaN,1173.0,department-of-defense,CONT_AWD_FA865016C7603_9700_-NONE-_-NONE-


In [7]:
contracts.shape

(100, 21)

In [9]:
capital = 10000000
portfolio = {}
valuation = []

In [20]:
def setLinkIntd (ticker, start, end, freq):
    link="https://financialmodelingprep.com/api/v3/historical-chart/"+freq+"/"+ticker+"?from="+start+"&to="+end+"&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"
    return link

def setLinkEod (ticker, start, end):
    return"https://financialmodelingprep.com/api/v3/historical-price-full/"+ticker+"?from="+start+"&to="+end+"&apikey=26srycwxWrFIhEuaZwic6mBdx7f4VjGT"

In [21]:
response = requests.get(url=setLinkIntd("LMT", "2010-03-31", "2010-03-31", "5min"))
response1 = requests.get(url=setLinkEod("RTX", "2001-01-01", "2001-01-02"))

In [27]:
response1.json()['historical']
prices = pd.Series([entry['close'] for entry in response1.json()['historical']])        
print(prices)

0    22.19
dtype: float64


In [14]:
from dataclasses import dataclass

@dataclass
class transaction:
    contractID: int
    num: int

In [15]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

def getBuyDay(contractDate):
    uBday = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    contractDate = pd.Timestamp(contractDate)
    buyDay = contractDate - 3 * uBday
    return buyDay.strftime('%Y-%m-%d')


In [16]:
def meanReversion(df):
    return

In [28]:
def getSellDay(contractDate, ticker):
    us_business_day = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    
    contractDate = pd.Timestamp(contractDate)
    adjDate = contractDate if us_business_day.is_on_offset(contractDate) else contractDate + us_business_day
    adjDate += 16 * us_business_day  
    
    startDay = adjDate - pd.Timedelta(days=30)
    endDay = adjDate
    
    response = requests.get(setLinkIntd(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d')), frequency="5min"))
    
    if response.status_code == 200:
        data = response.json()
        if len(data)==0: 
            try:
                response = requests.get(setLinkEod(ticker, startDay.strftime('%Y-%m-%d'), endDay.strftime(('%Y-%m-%d'))))
                data = response.json()['historical']
            except Exception as e: 
                print("Unexpected exception: "+ str(e))
                return None
        if len(data)!= 0:
            prices = pd.Series([entry['close'] for entry in data])        
            
            if meanReversion(prices):
                return adjDate.strftime('%Y-%m-%d')  
            else:
                next_day = adjDate + us_business_day
                return getSellDay(next_day.strftime('%Y-%m-%d'), ticker)  
        else: 
            return
    else:
        print(f"Error fetching data: {response.status_code}")
        return None


In [30]:
def buy(ticker, valuation, buyDate, id): 
    global portfolio, capital
    response = requests.get(url=setLinkIntd(ticker, buyDate, buyDate, "5min"))

    if response.status_code == 200: 
        if len(response.json()) == 0: 
            response = requests.get(url=setLinkEod(ticker, buyDate, buyDate))
            try: 
                pps = response.json()['historical']['close']
            except Exception as e: 
                print("Unexpected exception: "+ str(e))
                return None
        else:
            pps = response.json()[-15]["open"]
            shares = math.floor(valuation/pps)
            if ticker not in portfolio: 
                portfolio[ticker]= [transaction(contractID=id, num=shares)] 
            else: 
                portfolio[ticker].append(transaction(contractID=id, num=shares))
            capital-=shares*pps 
            print("purchased "+ str(shares) + " shares of "+ticker+" for "+ str(shares*pps)+". current portfolio valued at "+ str(capital))
            

In [134]:
def sell(ticker, sellDate, id):
    global portfolio, capital, valuation
    response = requests.get(url=setLink(ticker, sellDate, sellDate, "5min"))
    if response.status_code == 200 and len(response.json()) >15: 
        pps = response.json()[-15]["open"]
        for t in portfolio[ticker]: 
            if t.contractID == id: 
                shares = t.num
                value = shares*pps
                capital+=value 
                portfolio[ticker].remove(t)
                valuation.append(capital)
                print("sold "+str(shares) +" shares of " + ticker + " for " + str(value)+". current portfolio is valued at " + str(capital))
                break

In [ ]:
global capital

for i in range(200): 
    buy(contracts.iloc[i]['Recipient Name'], capital*0.1, getBuyDay(contracts.iloc[i]['Start Date']), contracts.iloc[i]['internal_id'])
    sell(contracts.iloc[i]['Recipient Name'], getSellDay(contracts.iloc[i]['Start Date']), contracts.iloc[i]['internal_id'])

purchased 1475 shares of GD for 99990.25000000001. current portfolio valued at 9900009.75
sold 1475 shares of GD for 103235.24999999999. current portfolio is valued at 10003245.0
purchased 1788 shares of RTN for 100020.72. current portfolio valued at 9903224.28
sold 1788 shares of RTN for 101522.64. current portfolio is valued at 10004746.92
purchased 1182 shares of LMT for 99973.56. current portfolio valued at 9904773.36
sold 1182 shares of LMT for 100824.59999999999. current portfolio is valued at 10005597.959999999
purchased 1734 shares of NOC for 99999.78. current portfolio valued at 9905598.18
sold 1734 shares of NOC for 99115.43999999999. current portfolio is valued at 10004713.62
purchased 2290 shares of CACI for 100004.3. current portfolio valued at 9904709.319999998
sold 2290 shares of CACI for 101905.0. current portfolio is valued at 10006614.319999998
purchased 2303 shares of CACI for 100065.35. current portfolio valued at 9906548.969999999
sold 2303 shares of CACI for 10446

KeyError: 'BLH'